# EECS 598 Project: Getting Started

This notebook will take you through the setup involved in the EECS 598 term project. We will also become familiar with basic SQL queries.

## 1. Install packages

Now launch Julia (either from your application launcher, or within a terminal). We're going to add some packages we will need later on. Type a closing square bracket ( `]` ) to enter pkg mode. Now type the following commands:

```julia
# type ] first to enter pkg mode (blue prompt), then do:
add LibPQ
add DataStreams
add DataFrames
add Plots
add CSV

# precompile all packages now so they load faster later:
precompile

# now backspace to exit pkg mode (prompt will return to green)
```

## 2. Connect to baseball database

The file `postgresql.jl` includes functions for connecting to the remote databases that contain baseball data:

In [1]:
include("postgresql.jl")

stream_to_dataframe (generic function with 1 method)

We've just loaded three connection functions for you to use, depending on which database your team will be working with:

```julia
# pitch data
connect_missing_pitch(
    connection_dir,
    username, 
    password
)

# probability of an out data
connect_probability_out(
    connection_dir,
    username, 
    password
)

# pitch tunneling data
connect_pitch_tunneling(
    connection_dir,
    username,
    password
)
```

Note that each database has its own directory with unique connection files -- be careful not to enter the wrong database path! Here are the three paths to choose from:

* Probability of an out: `./postgresql-files/probability-out/`
* Pitch tunneling: `./postgresql-files/pitch-tunneling/`
* Missing pitch imputation: `./postgresql-files/missing-pitch`

Let's connect to the "missing-pitch" database as an example, using an example username and password. You will need to enter the username and password combination for your group, along with the appropriate connection directory (which contains certificates and an authentication key):

In [2]:
connection_dir = "./postgresql-files/missing-pitch"

# enter your group's username and password as strings here:
user, pw = open("pw_missing_pitch.txt") do file
    readlines(file)
end

username = user
password = pw

conn = connect_missing_pitch(connection_dir, username, password)

Connecting to missing-pitch database.
Data are stored in the 'pitchmissing' table.


PostgreSQL connection (CONNECTION_OK) with parameters:
  user = user1
  password = ********************
  dbname = missing-pitch
  host = 141.211.55.211
  port = 58420
  client_encoding = UTF8
  application_name = LibPQ.jl
  sslmode = verify-ca
  sslcompression = 1
  sslcert = ./postgresql-files/missing-pitch/postgresql.crt
  sslkey = ./postgresql-files/missing-pitch/postgresql.key
  sslrootcert = ./postgresql-files/missing-pitch/root.crt
  krbsrvname = postgres
  target_session_attrs = any

## 3. Query databases

Note that the function printed the name of the relevant data table: `pitchmissing`. Having successfully initiated a connection, we can now send SQL queries to these tables. Let's ask for the list of columns in the `pitchmissing` table:

In [3]:
query = """
  SELECT ordinal_position,
         column_name,
         data_type,
         numeric_precision
    FROM information_schema.columns
   WHERE table_name = 'pitchmissing'
ORDER BY ordinal_position;
"""
result = execute(conn, query)

PostgreSQL result

(Here we queried a special table, the [information schema](https://www.postgresql.org/docs/9.1/static/information-schema.html). You don't need to worry about the details here -- just know that we can request metadata about tables using this schema.)

Okay, so the `execute` function returned a "PostgreSQL result" object -- but where's the data? At this point, we've told the server what we want, and it's ready to send it over. Now we ned to tell Julia to stream it in and store it somewhere locally:

In [4]:
data = Data.stream!(result, Data.Table)

(ordinal_position = Union{Missing, Int32}[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  34, 35, 36, 37, 38, 39, 40, 41, 42, 43], column_name = Union{Missing, String}["pitchguid", "gamepk", "batterid", "pitcherid", "pitchtype", "venueid", "islhp", "islhb", "inning", "istop"  …  "trajectorypolynomialz1", "trajectorypolynomialz2", "trajectoryverticalapproachangle", "trajectoryhorizontalapproachangle", "trajectoryverticalbreak", "trajectorylocationx", "trajectorylocationy", "trajectorylocationz", "atbatnumber", "pitchnumber"], data_type = Union{Missing, String}["character varying", "integer", "integer", "integer", "character varying", "integer", "bit", "bit", "integer", "bit"  …  "double precision", "double precision", "double precision", "double precision", "double precision", "double precision", "double precision", "double precision", "integer", "integer"], numeric_precision = Union{Missing, Int32}[missing, 32, 32, 32, missing, 32, missing, missing, 32, missing  …  53, 53, 53, 53, 53, 53, 53, 53, 32

Great! Now we've streamed a bunch of data. It looks ugly though. Let's wrap it up in a nice tidy DataFrame:

In [5]:
DataFrame(data)

,ordinal_position,column_name,data_type,numeric_precision
,Int32⍰,String⍰,String⍰,Int32⍰
1,1,pitchguid,character varying,missing
2,2,gamepk,integer,32
3,3,batterid,integer,32
4,4,pitcherid,integer,32
5,5,pitchtype,character varying,missing
6,6,venueid,integer,32
7,7,islhp,bit,missing
8,8,islhb,bit,missing
9,9,inning,integer,32


Much better. We can see the list of columns in the table, ready to be queried. That was a lot of steps just to get a dataframe! That's why we included a simple function in `postgresql.jl` to make things more convenient: `stream_to_dataframe()`.

If you don't want to think about SQL anymore, we'll show you the simplest query. This next query takes all data for the whole table and streams it over. Keep in mind that this will take up a significant chunk of memory (1.3 million rows and 43 columns!), and this approach won't always work in the real world, where datasets can be much larger. Be prepared to wait a while for this cell to run -- the kernel might even crash!

In [ ]:
# WARNING: takes a long time to run! May crash your kernel!
query = """
SELECT *
FROM pitchmissing;
"""

data = stream_to_dataframe(conn, query)

In [ ]:
size(data) # pretty big

If you were able to stream the whole dataset over, you can save it locally as a CSV:

In [ ]:
using CSV, DataFrames
CSV.write("pitch-missing.csv", data)

## 4. Learn more SQL! (optional)

You probably won't need the whole dataset in your analysis. You can use SQL queries to request specific data. There are many resources out there for learning about SQL queries. [Here](https://www.w3schools.com/sql/sql_intro.asp) is a good one.

**The rest of this notebook just provides more examples of SQL queries to help you work with these large datasets.**

### Taking a subset of rows from a subset of columns

Suppose we want the first 20 rows from the "pitcherid" and "pitchtype" columns. We can set up the query, execute, and stream as follows:

In [3]:
query = """
  SELECT pitcherid,
         pitchtype
    FROM pitchmissing
   LIMIT 20
"""

data = stream_to_dataframe(conn, query)

,pitcherid,pitchtype
,Int32⍰,String⍰
1,456027,FF
2,543506,FF
3,543272,SL
4,451085,SL
5,573124,FF
6,624586,FF
7,547179,FC
8,501817,FC
9,518886,KC


Why `LIMIT`? Because there are 1.3 million rows! Without LIMIT, the server would send us the entire column, and we don't need that (right now at least).

### Using GROUP BY to request an aggregated summary of data

Now let's group the data by pitcher ID and pitchtype, to see the most prolific pitchers and the kinds of throws they prefer:

In [4]:
query = """
SELECT pitcherid, pitchtype, COUNT(pitchtype)
FROM pitchmissing
GROUP BY pitcherid, pitchtype
ORDER BY COUNT(pitchtype) DESC
LIMIT 20;
"""

data = stream_to_dataframe(conn, query)

,pitcherid,pitchtype,count
,Int32⍰,String⍰,Int64⍰
1,434378,FF,4177
2,285079,KN,3972
3,592332,FF,3610
4,543699,FT,3313
5,453286,FF,3311
6,453562,SI,3278
7,592662,FF,3200
8,519043,FF,3150
9,462136,FF,3137


So the pitcher with ID 434378 threw a lot of four-seam fastballs. What is the pitch speed distribution for this pitcher?

In [5]:
query = """
SELECT pitchtype, ReleaseSpeed
FROM pitchmissing
WHERE pitcherid=434378;
"""

data = stream_to_dataframe(conn, query)

,pitchtype,releasespeed
,String⍰,Float64⍰
1,FF,95.4432
2,FF,96.5757
3,CU,79.588
4,FF,93.2888
5,FF,93.6737
6,FF,93.6486
7,FF,93.4014
8,SL,84.6488
9,FF,93.0501


Now let's visualize this pitcher's pitch type distribution:

In [6]:
using Plots
gr(
    label="",
    markerstrokewidth=0.3,
    markerstrokecolor="white",
    markersize=5,
)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
┌ Info: Precompiling GR [28b8d3ca-fb5f-59d9-8090-bfdbd6d07a71]
└ @ Base loading.jl:1187


Plots.GRBackend()

In [7]:
# use the project prompt pdf from Canvas to make a lookup dictionary
# for pitch types:
pitchtype_lookup = Dict(
  "CH" => "Changeup",
  "CU" => "Curveball",
  "FC" => "Cutter",
  "FF" => "Four-seam fastball",
  "FO" => "Forkball",
  "FS" => "Splitter",
  "FT" => "Two-seam fastball",
  "KC" => "Knuckle curve",
  "KN" => "Knuckleball",
  "SC" => "Screwball",
  "SI" => "Sinker",
  "SL" => "Slider"
  )

x = map(s -> pitchtype_lookup[s], data[:pitchtype])

y = data[:releasespeed]
scatter(x, y;
    xlabel="Pitch type", 
    ylabel="Speed (MPH)",
    title="Pitch speeds for pitcher 434378 ($(length(x)) pitches)"
)

ErrorException: No user recipe defined for Float64

### Comparing release speed, release angle, and height above plate

Let's find out more about pitch types. What are the typical measurements for a fastball vs a slider?

In [8]:
query = """
SELECT  pitchtype,
        COUNT(pitchtype),
        AVG(ReleaseSpeed) AS ReleaseSpeed_avg,
        AVG(ReleaseAngle) AS ReleaseAngle_avg,
        AVG(TrajectoryLocationZ) AS PlateHeight_avg
FROM pitchmissing
GROUP BY pitchtype
ORDER BY ReleaseSpeed_avg DESC
"""

data = stream_to_dataframe(conn, query)

,pitchtype,count,releasespeed_avg,releaseangle_avg,plateheight_avg
,String⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰
1,FF,483081,93.1343,-2.36179,2.60144
2,FT,191975,92.489,-1.94249,2.38947
3,SI,82901,91.5922,-1.47106,2.34229
4,FC,64369,88.4254,-1.72802,2.26584
5,FO,414,87.3514,-2.60116,1.45076
6,FS,20364,84.7098,-1.70737,1.74846
7,SL,202756,84.6036,-1.08401,1.92016
8,CH,138485,84.1588,-1.58601,1.89749
9,KC,31366,80.5894,0.396605,3.13352


In [9]:
pitchtype = data[:pitchtype]
releasespeed = data[:releasespeed_avg]
releaseangle = data[:releaseangle_avg]
plateheight = data[:plateheight_avg]

p1 = scatter(pitchtype, releasespeed; ylabel="Avg release speed (MPH)")
p2 = scatter(pitchtype, releaseangle; ylabel="Avg release angle (deg)")
p3 = scatter(pitchtype, plateheight; ylabel="Avg height above plate (ft)")
plot(p1, p2, p3; layout=(3, 1), size=(500, 700))

ErrorException: No user recipe defined for String

Glancing at our missing pitch prompt PDF, we see that ReleaseAngle represents the vertical angle of pitch release. The chart illustrates what we might expect: the fast a ball is released, the lower the angle should be in order for it to end up right over the home plate. You may be surprised to see that fastballs are released with a significant downward angle -- they travel so fast that they can make it all the way to the plate without dropping too far!

As a sanity check, look at the [forkball](https://en.wikipedia.org/wiki/Forkball) (FO). It has a very low release angle, and is not released very quickly, so we expect it to be quite low when it reaches the home plate. Sure enough, the bottom chart shows that forkballs end up 1.5 ft above the plate on average.

### Looking up missing values

Run the following cell to look up pitches for which the ReleaseSpeed value is missing:

In [ ]:
query = """
SELECT pitcherid, pitchtype, releasespeed, releaseangle
FROM pitchmissing
WHERE releasespeed IS NULL;
"""

data = stream_to_dataframe(conn, query)

In [ ]:
size(data)

For groups working on the pitch-missing project, these 1324 missing values are among those you will be trying to fill in! There are of course other similar commands for looking up missing values, but we have to leave some work for you to do :)